In [212]:
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pingouin import cronbach_alpha # cronbach alpha
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import mean_squared_error as MSE
from scipy.stats import pearsonr # pearson correlation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from factor_analyzer.factor_analyzer import (
    calculate_kmo,
) 
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import plotly_express as px
from math import sqrt

pd.options.display.float_format = "{:.4f}".format

def pearsonr_pval(x, y):
    return pearsonr(x, y)[1]

data = pd.read_csv("../app/data/combined_data2.csv")
info_data = data[["tract", "GEOID", "county"]]
data = data.drop(columns=["GEOID", "tract", "county", 'number_providers'], axis=1)


cv = StratifiedKFold(n_splits=10, shuffle=True)

In [213]:
data.has_computer = data.has_computer / data.total_population
data.with_internet = data.with_internet / data.total_population
data.work_from_home = data.work_from_home / data.total_population
data.broadband = data.broadband / data.total_population
data = data.drop("total_population", axis=1)

In [214]:
data

,has_computer,with_internet,work_from_home,broadband,median_income,median_age,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms
0,0.3852,0.3632,0.0336,0.3613,73942.0000,50.1000,58.0000,110.3274,41.7821,0.0517
1,0.3937,0.3743,0.0885,0.3687,72988.0000,53.9000,60.0769,145.7848,22.8873,0.0246
2,0.3521,0.3182,0.0017,0.3182,38077.0000,34.5000,60.0000,173.4079,27.0620,0.0300
3,0.3739,0.3198,0.0115,0.2908,38777.0000,38.9000,62.5000,127.6005,23.5383,0.0222
4,0.3824,0.3406,0.0074,0.3389,42639.0000,36.8000,62.5000,192.4299,21.6857,0.0217
...,...,...,...,...,...,...,...,...,...,...
178,0.2011,0.2068,0.0019,0.2068,39928.0000,34.1000,53.0000,88.0868,13.2451,0.0602
179,0.2325,0.2116,0.0107,0.2116,58884.0000,29.8000,53.0000,87.0108,17.0366,0.0444
180,0.2418,0.2216,0.0438,0.2216,50915.0000,29.8000,53.0000,74.1418,14.2204,0.0379
181,0.2268,0.2179,0.0073,0.2179,63750.0000,25.7000,52.2222,267.8397,32.9302,0.0238


In [215]:
pd.DataFrame(calculate_kmo(data)[0], index = data.columns)

,0
has_computer,0.9107
with_internet,0.6650
work_from_home,0.5011
broadband,0.6766
median_income,0.9556
median_age,0.8186
mean_lowest_cost,0.6816
mean_d_mbps,0.6539
mean_u_mbps,0.8200
mean_lat_ms,0.6732


In [216]:
calculate_kmo(data)[1]

0.7258436542979599

In [217]:
scaler = StandardScaler().fit(data)
data = pd.DataFrame(scaler.transform(data), columns=scaler.feature_names_in_)
pca = PCA().fit(data)
pca_data = pd.DataFrame(pca.transform(data), columns=pca.get_feature_names_out())

data = pd.concat([data.reset_index(drop=True), pca_data["pca0"].reset_index(drop=True)], axis=1).rename({"pca0": "index"}, axis=1)

In [218]:
# eigenvectors
pd.DataFrame(pca.components_)

,0,1,2,3,4,5,6,7,8,9
0,-0.4916,-0.5175,-0.0014,-0.5187,-0.1995,-0.0729,-0.1403,-0.2519,-0.1538,0.2636
1,-0.1961,-0.1399,-0.1962,-0.1307,-0.2089,-0.4996,0.1099,0.4774,0.3747,-0.4613
2,-0.0718,0.0088,0.5134,0.0080,-0.4857,0.1965,0.6185,0.0982,-0.2244,-0.1191
3,-0.0520,-0.0186,0.7548,-0.0089,0.1021,-0.0524,-0.5839,0.1681,0.0488,-0.2057
4,0.1834,0.1447,-0.2186,0.1424,-0.8023,0.0880,-0.4708,-0.0418,-0.0513,0.0018
5,0.0575,-0.0012,0.2074,-0.0015,-0.1419,0.0691,0.1161,-0.3228,0.8594,0.2693
6,0.1406,0.1582,0.1865,0.1642,-0.0772,-0.8275,0.0859,-0.2230,-0.1978,0.3265
7,-0.0578,0.0135,0.0144,0.0412,-0.0226,0.0761,-0.0247,0.7098,0.0430,0.6942
8,0.8070,-0.3849,0.0493,-0.4287,-0.0046,-0.0272,0.0405,0.1047,-0.0314,-0.0018
9,-0.0270,0.7196,0.0035,-0.6936,0.0005,-0.0050,-0.0041,0.0143,0.0039,0.0104


In [219]:
pd.DataFrame(pca.get_covariance())

,0,1,2,3,4,5,6,7,8,9
0,1.0055,0.9575,0.0139,0.9525,0.3478,0.3177,0.1131,0.1401,0.1015,-0.1746
1,0.9575,1.0055,0.0504,1.0039,0.3281,0.2590,0.1804,0.2577,0.1194,-0.2795
2,0.0139,0.0504,1.0055,0.0534,0.0305,0.2366,-0.0566,-0.1129,-0.1537,0.0776
3,0.9525,1.0039,0.0534,1.0055,0.3265,0.2451,0.1784,0.2738,0.1289,-0.2891
4,0.3478,0.3281,0.0305,0.3265,1.0055,0.1767,-0.0541,-0.0480,-0.0183,0.0588
5,0.3177,0.2590,0.2366,0.2451,0.1767,1.0055,-0.0065,-0.4496,-0.3677,0.3980
6,0.1131,0.1804,-0.0566,0.1784,-0.0541,-0.0065,1.0055,0.1977,0.0848,-0.1821
7,0.1401,0.2577,-0.1129,0.2738,-0.0480,-0.4496,0.1977,1.0055,0.4066,-0.8361
8,0.1015,0.1194,-0.1537,0.1289,-0.0183,-0.3677,0.0848,0.4066,1.0055,-0.4235
9,-0.1746,-0.2795,0.0776,-0.2891,0.0588,0.3980,-0.1821,-0.8361,-0.4235,1.0055


In [220]:
pd.DataFrame(pca.components_).head()

,0,1,2,3,4,5,6,7,8,9
0,-0.4916,-0.5175,-0.0014,-0.5187,-0.1995,-0.0729,-0.1403,-0.2519,-0.1538,0.2636
1,-0.1961,-0.1399,-0.1962,-0.1307,-0.2089,-0.4996,0.1099,0.4774,0.3747,-0.4613
2,-0.0718,0.0088,0.5134,0.0080,-0.4857,0.1965,0.6185,0.0982,-0.2244,-0.1191
3,-0.0520,-0.0186,0.7548,-0.0089,0.1021,-0.0524,-0.5839,0.1681,0.0488,-0.2057
4,0.1834,0.1447,-0.2186,0.1424,-0.8023,0.0880,-0.4708,-0.0418,-0.0513,0.0018


In [221]:
list(pca.explained_variance_)[0:7]

[3.436434190186439,
 2.5079970962190368,
 1.0263476578142086,
 0.9779124049477561,
 0.7622902984964774,
 0.674281847306549,
 0.45108231612062116]

In [222]:
list(pca.explained_variance_ratio_.cumsum())[0:7]

[0.34176558612783176,
 0.5911948055332223,
 0.6932687255999906,
 0.7905255877314068,
 0.8663380655053734,
 0.9333977902101778,
 0.9782595287533216]

In [223]:
data.corr()

,has_computer,with_internet,work_from_home,broadband,median_income,median_age,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms,index
has_computer,1.0000,0.9522,0.0139,0.9473,0.3459,0.3160,0.1125,0.1394,0.1009,-0.1737,-0.9088
with_internet,0.9522,1.0000,0.0501,0.9984,0.3263,0.2576,0.1794,0.2563,0.1187,-0.2780,-0.9568
work_from_home,0.0139,0.0501,1.0000,0.0531,0.0304,0.2353,-0.0563,-0.1122,-0.1529,0.0772,-0.0027
broadband,0.9473,0.9984,0.0531,1.0000,0.3247,0.2438,0.1774,0.2723,0.1282,-0.2875,-0.9589
median_income,0.3459,0.3263,0.0304,0.3247,1.0000,0.1757,-0.0538,-0.0477,-0.0182,0.0585,-0.3689
median_age,0.3160,0.2576,0.2353,0.2438,0.1757,1.0000,-0.0064,-0.4471,-0.3657,0.3958,-0.1348
mean_lowest_cost,0.1125,0.1794,-0.0563,0.1774,-0.0538,-0.0064,1.0000,0.1966,0.0844,-0.1811,-0.2593
mean_d_mbps,0.1394,0.2563,-0.1122,0.2723,-0.0477,-0.4471,0.1966,1.0000,0.4044,-0.8316,-0.4658
mean_u_mbps,0.1009,0.1187,-0.1529,0.1282,-0.0182,-0.3657,0.0844,0.4044,1.0000,-0.4211,-0.2843
mean_lat_ms,-0.1737,-0.2780,0.0772,-0.2875,0.0585,0.3958,-0.1811,-0.8316,-0.4211,1.0000,0.4874


In [224]:
data.corr(pearsonr_pval)

,has_computer,with_internet,work_from_home,broadband,median_income,median_age,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms,index
has_computer,1.0000,0.0000,0.8524,0.0000,0.0000,0.0000,0.1294,0.0599,0.1741,0.0187,0.0000
with_internet,0.0000,1.0000,0.5002,0.0000,0.0000,0.0004,0.0151,0.0005,0.1094,0.0001,0.0000
work_from_home,0.8524,0.5002,1.0000,0.4751,0.6833,0.0013,0.4491,0.1303,0.0388,0.2990,0.9713
broadband,0.0000,0.0000,0.4751,1.0000,0.0000,0.0009,0.0163,0.0002,0.0837,0.0001,0.0000
median_income,0.0000,0.0000,0.6833,0.0000,1.0000,0.0174,0.4697,0.5211,0.8072,0.4313,0.0000
median_age,0.0000,0.0004,0.0013,0.0009,0.0174,1.0000,0.9311,0.0000,0.0000,0.0000,0.0688
mean_lowest_cost,0.1294,0.0151,0.4491,0.0163,0.4697,0.9311,1.0000,0.0076,0.2562,0.0142,0.0004
mean_d_mbps,0.0599,0.0005,0.1303,0.0002,0.5211,0.0000,0.0076,1.0000,0.0000,0.0000,0.0000
mean_u_mbps,0.1741,0.1094,0.0388,0.0837,0.8072,0.0000,0.2562,0.0000,1.0000,0.0000,0.0001
mean_lat_ms,0.0187,0.0001,0.2990,0.0001,0.4313,0.0000,0.0142,0.0000,0.0000,1.0000,0.0000


In [225]:
data = data.drop(["work_from_home", "median_age"], axis=1)

In [226]:
data.corr(pearsonr_pval)

,has_computer,with_internet,broadband,median_income,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms,index
has_computer,1.0000,0.0000,0.0000,0.0000,0.1294,0.0599,0.1741,0.0187,0.0000
with_internet,0.0000,1.0000,0.0000,0.0000,0.0151,0.0005,0.1094,0.0001,0.0000
broadband,0.0000,0.0000,1.0000,0.0000,0.0163,0.0002,0.0837,0.0001,0.0000
median_income,0.0000,0.0000,0.0000,1.0000,0.4697,0.5211,0.8072,0.4313,0.0000
mean_lowest_cost,0.1294,0.0151,0.0163,0.4697,1.0000,0.0076,0.2562,0.0142,0.0004
mean_d_mbps,0.0599,0.0005,0.0002,0.5211,0.0076,1.0000,0.0000,0.0000,0.0000
mean_u_mbps,0.1741,0.1094,0.0837,0.8072,0.2562,0.0000,1.0000,0.0000,0.0001
mean_lat_ms,0.0187,0.0001,0.0001,0.4313,0.0142,0.0000,0.0000,1.0000,0.0000
index,0.0000,0.0000,0.0000,0.0000,0.0004,0.0000,0.0001,0.0000,1.0000


In [227]:
data = data.drop(["index"], axis=1)

In [228]:
pca2 = PCA().fit(data)

In [229]:
pd.DataFrame(calculate_kmo(data)[0], index = data.columns)

,0
has_computer,0.9141
with_internet,0.6554
broadband,0.6713
median_income,0.9516
mean_lowest_cost,0.7041
mean_d_mbps,0.5989
mean_u_mbps,0.8168
mean_lat_ms,0.6253


In [230]:
calculate_kmo(data)[1]

0.7045665299835115

In [231]:
pca_data2  = pd.DataFrame(pca2.transform(data), columns=pca2.get_feature_names_out())

In [232]:
data = pd.concat([data.reset_index(drop=True), pca_data2["pca0"].reset_index(drop=True)], axis=1).rename({"pca0": "index"}, axis=1)

In [233]:
X = data.drop(["index"], axis=1)
y = data["index"]

lr = LinearRegression()

sfs = SFS(
    lr,
    forward=True,
    floating=True,
    scoring="r2",
    cv=10,
    k_features="parsimonious"
)

sfs.fit(X, y)

SequentialFeatureSelector(cv=10, estimator=LinearRegression(), floating=True,
                          k_features=(1, 8), scoring='r2')

In [234]:
pd.DataFrame(sfs.get_metric_dict()).T

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(2,)","[0.4928268127466475, 0.911811010763889, 0.9398...",0.7421,"(broadband,)",0.1452,0.1956,0.0652
2,"(2, 7)","[0.8595238174447366, 0.9295018812338532, 0.977...",0.9324,"(broadband, mean_lat_ms)",0.0266,0.0358,0.0119
3,"(2, 6, 7)","[0.9079273670370004, 0.9429963742648841, 0.980...",0.9550,"(broadband, mean_u_mbps, mean_lat_ms)",0.0162,0.0218,0.0073
4,"(0, 2, 6, 7)","[0.9424478343956092, 0.9475262572862382, 0.988...",0.9630,"(has_computer, broadband, mean_u_mbps, mean_la...",0.0113,0.0153,0.0051
5,"(0, 2, 4, 6, 7)","[0.9780726141635088, 0.9404701865141804, 0.988...",0.9746,"(has_computer, broadband, mean_lowest_cost, me...",0.0104,0.0140,0.0047
6,"(0, 2, 4, 5, 6, 7)","[0.9917697142318903, 0.9410022360020907, 0.993...",0.9898,"(has_computer, broadband, mean_lowest_cost, me...",0.0122,0.0164,0.0055
7,"(0, 1, 2, 4, 5, 6, 7)","[0.9966079534421284, 0.9411498546067251, 0.994...",0.9907,"(has_computer, with_internet, broadband, mean_...",0.0123,0.0166,0.0055
8,"(0, 1, 2, 3, 4, 5, 6, 7)","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0000,"(has_computer, with_internet, broadband, media...",0.0000,0.0000,0.0000


In [235]:
list(sfs.get_metric_dict()[4]["feature_names"])

['has_computer', 'broadband', 'mean_u_mbps', 'mean_lat_ms']

In [236]:
list(sfs.get_metric_dict()[5]["feature_names"])

['has_computer', 'broadband', 'mean_lowest_cost', 'mean_u_mbps', 'mean_lat_ms']

In [237]:
X[list(sfs.get_metric_dict()[6]["feature_names"])]

,has_computer,broadband,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms
0,0.1655,0.2161,-0.1851,-1.0620,0.2839,1.1831
1,0.2911,0.3251,0.4204,-0.6423,-0.4316,-0.1546
2,-0.3240,-0.4260,0.3980,-0.3153,-0.2735,0.1099
3,-0.0019,-0.8333,1.1268,-0.8576,-0.4070,-0.2762
4,0.1247,-0.1175,1.1268,-0.0902,-0.4771,-0.3011
...,...,...,...,...,...,...
178,-2.5534,-2.0832,-1.6427,-1.3253,-0.7968,1.6012
179,-2.0895,-2.0124,-1.6427,-1.3381,-0.6532,0.8235
180,-1.9520,-1.8640,-1.6427,-1.4904,-0.7598,0.5008
181,-2.1746,-1.9188,-1.8695,0.8025,-0.0513,-0.1935


In [238]:
X_new = X[list(sfs.get_metric_dict()[6]["feature_names"])]
X_new = sm.add_constant(X_new)

In [239]:
model = sm.OLS(y, X_new).fit()

In [240]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  index   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     3122.
Date:                Thu, 01 Jun 2023   Prob (F-statistic):          7.16e-176
Time:                        16:08:31   Log-Likelihood:                 56.093
No. Observations:                 183   AIC:                            -98.19
Df Residuals:                     176   BIC:                            -75.72
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             6.939e-18      0.013  

In [241]:
(-model.params)

const              -0.0000
has_computer        0.5482
broadband           1.0309
mean_lowest_cost    0.1315
mean_d_mbps         0.2661
mean_u_mbps         0.1694
mean_lat_ms        -0.2688
dtype: float64

In [242]:
variables = list(model.params.keys())
del variables[0]
variables

['has_computer',
 'broadband',
 'mean_lowest_cost',
 'mean_d_mbps',
 'mean_u_mbps',
 'mean_lat_ms']

In [243]:
coefs = list(-model.params)

In [244]:
del coefs[0]

In [245]:
coefs

[0.5482224427627346,
 1.0309023594736633,
 0.13151718981440325,
 0.26609847462880765,
 0.1693613062725399,
 -0.26884025773892756]

In [246]:
lr.fit(X_new, y)

LinearRegression()

In [247]:
coefficients = list(-lr.coef_)
del coefficients[0]

In [248]:
coefficients

[0.5482224427627367,
 1.0309023594736606,
 0.13151718981440425,
 0.26609847462880776,
 0.16936130627254017,
 -0.2688402577389286]

In [249]:
data = data[variables]

In [250]:
data["index"] = data.mul(coefficients).sum(axis=1)

In [251]:
data

,has_computer,broadband,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms,index
0,0.1655,0.2161,-0.1851,-1.0620,0.2839,1.1831,-0.2634
1,0.2911,0.3251,0.4204,-0.6423,-0.4316,-0.1546,0.3476
2,-0.3240,-0.4260,0.3980,-0.3153,-0.2735,0.1099,-0.7242
3,-0.0019,-0.8333,1.1268,-0.8576,-0.4070,-0.2762,-0.9347
4,0.1247,-0.1175,1.1268,-0.0902,-0.4771,-0.3011,0.0716
...,...,...,...,...,...,...,...
178,-2.5534,-2.0832,-1.6427,-1.3253,-0.7968,1.6012,-4.6815
179,-2.0895,-2.0124,-1.6427,-1.3381,-0.6532,0.8235,-4.1243
180,-1.9520,-1.8640,-1.6427,-1.4904,-0.7598,0.5008,-3.8677
181,-2.1746,-1.9188,-1.8695,0.8025,-0.0513,-0.1935,-3.1593


In [253]:
pd.concat([info_data, data], axis=1).to_csv("../app/data/FINAL_INDEX_DATA2.csv", index=False)